In [ ]:
%load_ext autoreload
%autoreload 2

import os
import mne
from mne.preprocessing import ICA
import multiprocessing as mpr
import numpy as np
import utils
import json
import matplotlib.pyplot as plt

import utils_preproc as upre
import matplotlib as mpl
import globvars as gv

import utils
import utils_test

In [ ]:
from os.path import join as pjoin

In [ ]:
rawname_ = 'S01_off_move'
fname_noext_def = rawname_ + '_resample_raw'
fname_noext_def = rawname_ + '_resample_notch_highpass_raw'

In [ ]:
fname_def = pjoin(gv.data_dir,fname_noext_def + '.fif')
f = mne.io.read_raw_fif(fname_def, None)
mod_info, infos = upre.readInfo(rawname_, f)
f.info = mod_info

In [ ]:
f.ch_names

### Info

In [ ]:
from globvars import gp

info_fname = 'definfo.npz'
#np.savez(info_fname, mod_info)

fi = np.load(info_fname,allow_pickle=1)

info = fi['arr_0'][()]

# Gather stats from exiting

In [ ]:
import utils_tSNE as utsne

In [ ]:
f['EMG062_old'][0].shape

In [ ]:
emgdat = f['EMG062_old'][0][0]
lfpdat = f['LFPL01'][0][0]
#Memgdat

emg_mean,emg_std = np.mean(emgdat), np.std(emgdat)
lfp_mean,lfp_std = np.mean(lfpdat), np.std(lfpdat)

emg_mean, emg_std, mn,mx = utsne.robustMean(emgdat,  ret_aux=1,
                                     ret_std=True, q = .05)
lfp_mean, lfp_std, mn,mx = utsne.robustMean(lfpdat,  ret_aux=1,
                                     ret_std=True, q = .05)




info = f.info
meg_chis = mne.pick_channels_regexp(info.ch_names,'MEG.*')

print(len(meg_chis))

#for chi in meg_chis:
megdat = f[meg_chis][0]
# meg_means, mn,mx = utsne.robustMean(megdat, axis=1, per_dim =1, ret_aux=1, q = .05)
# meg_vars, _,_ = utsne.robustMean((megdat - meg_means[:,None])**2, axis=1, per_dim =1, 
#                             ret_aux=1, q = .05)
# meg_stds = np.sqrt(meg_vars)

meg_means, meg_stds, mn,mx = utsne.robustMean(megdat, axis=1, per_dim =1, ret_aux=1,
                                     ret_std=True, q = .05)
#meg_means,meg_stds = np.mean(megdat,axis=1), np.std(megdat,axis=1)
meg_std = np.mean(meg_stds)
meg_mean = np.mean(meg_means)

print(meg_mean, meg_std)

In [ ]:
plt.plot(lfpdat)

In [ ]:
plt.plot(emgdat)

In [ ]:
plt.hist(meg_means)

In [ ]:
plt.hist(meg_stds)

# Set params

In [ ]:
trem_side = 'left'  # this is by default
sind_str_new = 'S99'

In [ ]:
trem_side = 'right'
sind_str_new = 'S97'

In [ ]:
emg_chnames_with_tremor = gv.EMG_per_hand[trem_side]
emg_chis = mne.pick_channels_regexp(info.ch_names,'EMG.*_old')
lfp_chis = mne.pick_channels_regexp(info.ch_names,'LFP.*')

glob_indices_emg = [info.ch_names.index(chn) for chn in emg_chnames_with_tremor]

tremsidelet = trem_side[0].upper()
opsidelet = utils.getOppositeSideStr(tremsidelet)
glob_index_lfp = [chi for chii,chi in enumerate(lfp_chis) if \
              info.ch_names[chi].startswith('LFP{}'.format(opsidelet)) ][0]

In [ ]:
task = None

In [ ]:
total_duration = 33 #in sec
sfreq = int(info['sfreq'])
sfreq_high = 1024

trem_intervals = [(10,20), (28,32)]
#trem_start, trem_end = 10,20
trem_start_trans = 3
trem_end_trans = 3

# EMG
trem_hifreq = 100
trem_lowfreq = 6

emg_chis_to_have_tremor = [emg_chis.index(glob_index_emg) for glob_index_emg in glob_indices_emg]   #among EMG channels

#LFP
trem_burst_lfp_freq = 70
artfiacts_intervals_LFP = [(6.5,6.7,0), (9,9.2,0), (12,12.1,1)]
lfp_chi_to_have_tremor = lfp_chis.index(glob_index_lfp)

#MEG
roi_labels_with_tremor = ['Supp_Motor_Area_{}'.format(opsidelet), 
                          'Cerebellum_{}'.format(tremsidelet)]
trem_burst_meg_freq = 40
artfiacts_intervals_MEG = [(6.4,6.6,0), (7,8,0), (14.5,15,1)]
#artfiacts_intervals_MEG = []
do_gen_MEG_data = False;

In [ ]:
10 * 256 /32

In [ ]:
tremsidelet

In [ ]:


trem_burst_lfp_freq
it_LFP = 'BAD_LFP{}'.format(opsidelet)
it_LFP_otherside = 'BAD_LFP{}'.format(tremsidelet)
it_MEG = 'BAD_MEG{}'.format(opsidelet)
it_notrem = 'notrem_{}'.format(tremsidelet)
it_trem = 'trem_{}'.format(tremsidelet)
it_trem_otherside = 'trem_{}'.format(opsidelet)
it_notrem_otherside = 'notrem_{}'.format(opsidelet)
it_move = 'move_{}'.format(tremsidelet)

move_intervals = [(3,6)]  # I dont' simulate it, just to test processing of intervals
notrem_intervals = [(0,3),(6,17),(23,26)]

onset = []
dur = []
its = []
for ait in artfiacts_intervals_MEG:
    onset += [ait[0]]
    dur += [ait[1] -ait[0]]
    its += [it_MEG]
MEGartif = mne.Annotations(onset,dur,its)


onset = []
dur = []
its = []
its_otherside = []
for ait in artfiacts_intervals_LFP:
    onset += [ait[0]]
    dur += [ait[1] - ait[0]]
    its += [it_LFP]
    its_otherside += [it_LFP_otherside]
LFPartif = mne.Annotations(onset,dur,its) + mne.Annotations(onset,dur,its_otherside)

#print(fn)

onset = []
dur = []
its = []
for ait in trem_intervals:
    onset += [ait[0]]
    dur += [ait[1]-ait[0]]
    its += [it_trem]
for ait in notrem_intervals:
    onset += [ait[0]]
    dur += [ait[1]-ait[0]]
    its += [it_notrem]
onset += [25]
dur += [4]
its += [it_notrem_otherside]
sh = 4
sh2 = 1.3
for ait in trem_intervals[ 0:1]:
    onset += [ait[0] + sh]
    dur += [ait[1] - ait[0] - sh2]
    its += [it_trem_otherside]
for ait in move_intervals:
    onset += [ait[0]]
    dur += [ait[1] - ait[0]]
    its += [it_move]

mvt = mne.Annotations(onset,dur,its) 


do_save_anns = True

pre_rns = ['{}_off_hold', '{}_off_move', '{}_on_move', '{}_on_hold']
for pre_rn in pre_rns:
    rn = pre_rn.format(sind_str_new)
    fn = pjoin(gv.data_dir,rn + '_ann_LFPartif.txt')
    if do_save_anns:
        LFPartif.save(fn)

    fn = pjoin(gv.data_dir,rn + '_ann_MEGartif.txt')
    if do_save_anns:
        MEGartif.save(fn)

    fn = pjoin(gv.data_dir,rn + '_anns.txt')
    if do_save_anns:
        mvt.save(fn)
    print('Saved anns {} for {}'.format(do_save_anns,rn) )
    
rawname_new = pre_rns[1].format(sind_str_new)

In [ ]:
mvt.description, mvt.duration

In [ ]:
rawname_new

In [ ]:
fn

In [ ]:
pre_rn.format(sind_str)

In [ ]:
rn

In [ ]:
sind_str

In [ ]:
mvt

In [ ]:
line_noise_freq = 50
line_noise_harmonics = np.arange(line_noise_freq, sfreq//2, line_noise_freq)
line_noise_harmonics_hires = np.arange(line_noise_freq, sfreq_high//2, line_noise_freq)
SNR_line_noise = 3
std2noise_size_mult = {'LFP': 0.1, 'EMG':0.1, 'MEG':0.1}
line_noise_scale_decay = 0.5

In [ ]:
times = np.arange(total_duration * sfreq) / sfreq
times_hires = np.arange(total_duration * sfreq_high) / sfreq_high


In [ ]:
#print(trem_side,  info.ch_names[glob_index_emg], info.ch_names[glob_index_lfp])

In [ ]:
# emg_mean,emg_std = (0.0006595533493318836, 1.7004551716638424e-05)  # from S01_off_move
# lfp_mean,lfp_std = (1.802609245871129e-05, 6.75905740440924e-06)
## does not make sense meg_mean, meg_std = (2.0509702638406777e-13, 3.328465797177012e-11)

In [ ]:
random_seed = 0
np.random.seed(random_seed)

In [ ]:
supp_info = {'random_seed': random_seed, 'trem_intervals':trem_intervals,
            'trem_start_trans':trem_start_trans, 'trem_end_trans':trem_end_trans,
            'trem_hifreq':trem_hifreq, 'trem_lowfreq':trem_lowfreq}

In [ ]:
supp_info['emg_chis_to_have_tremor'] = emg_chis_to_have_tremor
supp_info['trem_burst_lfp_freq'] = trem_burst_lfp_freq
supp_info['artfiacts_intervals_LFP'] = artfiacts_intervals_LFP
supp_info['lfp_chi_to_have_tremor'] = lfp_chi_to_have_tremor
supp_info['roi_labels_with_tremor'] = roi_labels_with_tremor
supp_info['trem_burst_meg_freq'] = trem_burst_meg_freq
supp_info['artfiacts_intervals_MEG'] = artfiacts_intervals_MEG
supp_info['meg_mean'] = meg_mean
supp_info['meg_std'] = meg_std
supp_info['meg_means'] = meg_means
supp_info['meg_stds'] = meg_stds
supp_info['lfp_mean'] = lfp_mean
supp_info['lfp_std'] = lfp_std
supp_info['emg_mean'] = emg_mean
supp_info['emg_std'] = emg_std
supp_info['trem_side'] = trem_side
supp_info['total_duration'] = total_duration
supp_info['sfreq'] = sfreq
supp_info['sfreq_high'] = sfreq_high

supp_info['rawname_base'] = rawname_
if task is None:
    supp_info['task_present'] = ''  #needed for correct saving to .mat file
else:
    supp_info['task_present'] = task

# generate EMG data

In [ ]:
#200 hz sine modulated

In [ ]:
tremor_intensity = np.zeros( len(times) )
tremor_intensity_hires = np.zeros( len(times_hires) )
for trem_start, trem_end in trem_intervals:
    tremor_intensity[trem_start*sfreq:trem_end*sfreq] += 1
    tremor_intensity[(trem_start - trem_start_trans)*sfreq: trem_start*sfreq] += \
        np.linspace(0,1,trem_start_trans*sfreq)  
    sl = slice(trem_end*sfreq, (trem_end + trem_end_trans)*sfreq)
    tremor_intensity[sl] += \
        np.linspace(1,0,min(trem_end_trans*sfreq,len(times)-trem_end*sfreq)  )
    
    tremor_intensity_hires[trem_start*sfreq_high:trem_end*sfreq_high] += 1
    tremor_intensity_hires[(trem_start - trem_start_trans)*sfreq_high: trem_start*sfreq_high] += \
        np.linspace(0,1,trem_start_trans*sfreq_high)  
    sl = slice(trem_end*sfreq_high, (trem_end + trem_end_trans)*sfreq_high)
    tremor_intensity_hires[sl] += \
        np.linspace(1,0,min(trem_end_trans*sfreq_high,len(times_hires)-trem_end*sfreq_high)  ) 

supp_info['tremor_intensity'] = tremor_intensity
plt.plot(times,tremor_intensity); plt.show()
plt.plot(times_hires,tremor_intensity_hires)

In [ ]:
emg_low, emg_high = emg_mean - emg_std, emg_mean + emg_std
emg_noise_mult = (emg_high - emg_low) * std2noise_size_mult['EMG']
EMG_noise  = np.random.uniform(low = -1, high = 1, size=len(times))
EMG_noise2 = np.random.uniform(low = -1, high = 1, size=len(times))


trem_signal_hifreq = np.sin(2*np.pi * trem_hifreq * times)
trem_signal_modulation = np.sin(2*np.pi * trem_lowfreq * times)
EMG_signal = emg_mean + (emg_high - emg_low)* trem_signal_modulation * trem_signal_hifreq * tremor_intensity 
EMG_signal2 = EMG_signal[:]
#EMG_signal  +=  EMG_noise * emg_noise_mult
#EMG_signal2 +=  EMG_noise2 * emg_noise_mult * 0.7

EMG_signals = [EMG_signal,EMG_signal2 ]

In [ ]:
nemgs = len(emg_chis)
emg_data = np.random.uniform(low = -1, high = 1, size=(nemgs,len(times)) )
emg_data *= emg_noise_mult
emg_data += emg_mean

for ii,emg_chi_to_have_tremor in enumerate(emg_chis_to_have_tremor):
    emg_data[emg_chi_to_have_tremor] += EMG_signals[ii]
print(nemgs,emg_data.shape)

for emg_chi_to_have_tremor in emg_chis_to_have_tremor:
    plt.figure(figsize=(20,3))
    plt.plot(times,emg_data[emg_chi_to_have_tremor])
    # plt.xlim(7,9)
    plt.title(info.ch_names[emg_chis[emg_chi_to_have_tremor]] )
    plt.xlim(5,13)
    plt.show()

In [ ]:
EMG_signals[0] - EMG_signals[1]

# generate LFP data

In [ ]:
lfp_low, lfp_high = lfp_mean - lfp_std, lfp_mean + lfp_std
lfp_noise_mult = (lfp_high - lfp_low) * std2noise_size_mult['LFP']
line_noise_ampl_LFP = lfp_noise_mult * SNR_line_noise

# let us have 70 Hz osc during tremor

trem_burst_lfp = np.sin(2*np.pi * trem_burst_lfp_freq * times)
LFP_signal = lfp_mean + (lfp_high - lfp_low)* trem_burst_lfp * tremor_intensity 

# add two artifacts 
for ai in artfiacts_intervals_LFP:
    start,end,at = ai
    start_bin,end_bin = int(start*sfreq), int(end*sfreq)
    if at == 0:
        art_signal = np.random.uniform(low = -1, high = 1, size=end_bin-start_bin) \
            * (lfp_high - lfp_low) * 4
    elif at == 1:
        art_signal = (lfp_high - lfp_low) * 4

    LFP_signal[start_bin:end_bin] += art_signal

In [ ]:
nlfps = len(lfp_chis)
lfp_data = np.random.uniform(low = -1, high = 1, size=(nlfps,len(times)) )
lfp_data *= lfp_noise_mult
lfp_data += lfp_mean

line_noise_scale = 1
for lnhi,lnh in enumerate(line_noise_harmonics):
    lfp_data += line_noise_scale * \
        line_noise_ampl_LFP * np.sin(2*np.pi * lnh * times) [None,:]
    line_noise_scale *= line_noise_scale_decay

lfp_data[lfp_chi_to_have_tremor] += LFP_signal # make the last one
print(info.ch_names[lfp_chis[lfp_chi_to_have_tremor] ] )

plt.figure(figsize=(20,3))
plt.plot(times,lfp_data[lfp_chi_to_have_tremor])
plt.show()

In [ ]:
#utils.getChannelSide(rawname_,emg_chis[0])

In [ ]:
lfp_data.shape

# Generate HFO LFP data

In [ ]:
lfp_low, lfp_high = lfp_mean - lfp_std, lfp_mean + lfp_std
lfp_noise_mult = (lfp_high - lfp_low) * std2noise_size_mult['LFP']
line_noise_ampl_LFP = lfp_noise_mult * SNR_line_noise

# let us have 70 Hz osc during tremor

trem_burst_lfp_hires = np.sin(2*np.pi * trem_burst_lfp_freq * times_hires)
LFP_signal_hires = lfp_mean + (lfp_high - lfp_low)* trem_burst_lfp_hires * \
tremor_intensity_hires

# add two artifacts 
for ai in artfiacts_intervals_LFP:
    start,end,at = ai
    start_bin,end_bin = int(start*sfreq_high), int(end*sfreq_high)
    if at == 0:
        art_signal = np.random.uniform(low = -1, high = 1, size=end_bin-start_bin) \
            * (lfp_high - lfp_low) * 4
    elif at == 1:
        art_signal = (lfp_high - lfp_low) * 4

    LFP_signal_hires[start_bin:end_bin] += art_signal

In [ ]:
nlfps = len(lfp_chis)
lfp_data_hires = np.random.uniform(low = -1, high = 1, size=(nlfps,len(times_hires)) )
lfp_data_hires *= lfp_noise_mult
lfp_data_hires += lfp_mean

line_noise_scale = 1
for lnhi,lnh in enumerate(line_noise_harmonics_hires):
    lfp_data_hires += line_noise_scale * \
        line_noise_ampl_LFP * np.sin(2*np.pi * lnh * times_hires) [None,:]
    line_noise_scale *= line_noise_scale_decay

lfp_data_hires[lfp_chi_to_have_tremor] += LFP_signal_hires # make the last one
print(info.ch_names[lfp_chis[lfp_chi_to_have_tremor] ] )

plt.figure(figsize=(20,3))
plt.plot(times_hires,lfp_data_hires[lfp_chi_to_have_tremor])
plt.show()

# generate MEG data

## Load coords

In [ ]:
sind_str,mod,task = utils.getParamsFromRawname(rawname_)

sources_type='parcel_aal'
src_file_grouping_ind = 10
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rawname_,
                                                    sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)


print( list(rec_info.keys()) )

labels_dict = rec_info['label_groups_dict'][()]
srcgroups_dict = rec_info['srcgroups_dict'][()]
coords = rec_info['coords_Jan_actual'][()]

print( labels_dict.keys() )

sgdn = 'all_raw'

In [ ]:
chan_info

In [ ]:
#3012 (FIFFV_COIL_VV_PLANAR_T1)
#2000 (FIFFV_COIL_POINT_MAGNETOMETER)

In [ ]:
chlocs = []
sensor_types = []
for si in range(len(meg_chis)):
    chi = meg_chis[si]
    chan_info = f.info['chs'][chi]
    chan_loc = chan_info['loc'][:3]
    chlocs += [chan_loc]
    #sensor_types += [chan_info['coil_type']._name]
    sensor_types += [int(chan_info['coil_type']) ]
    print(chan_info['coil_type'])
chlocs = np.vstack(chlocs)

In [ ]:
assert fname_noext_def.endswith('resample_raw'), fname_noext_def

## Put coherent info in sensors corresponding to different roi

In [ ]:
#lbls = [ 'Cerebellum_R']
# we get indices of sensor as they were presented in chlocs, i.e. in the order of meg_chis
roi_sensors = utils_test.getSensorsFromRoiLabels(roi_labels_with_tremor, 
                                                 coords, chlocs*100, 
                                                 labels_dict[sgdn], 
                                                 srcgroups_dict[sgdn] )
display(roi_sensors)
supp_info['roi_sensors'] = roi_sensors

### Gen data

In [ ]:
meg_noise_mults.shape, line_noise_ampl_MEGs.shape

In [ ]:
meg_means.shape

In [ ]:
nmegs = len(meg_chis)
meg_noise_mults = 2*meg_stds * std2noise_size_mult['MEG']
meg_data = np.random.uniform(low = -1, high = 1, size=(nmegs,len(times)) )
#meg_data *= meg_noise_mult
meg_data *= meg_noise_mults[:,None]
meg_data += meg_means[:,None]

line_noise_ampl_MEGs = meg_noise_mults * SNR_line_noise

line_noise_scale = 1
for lnhi,lnh in enumerate(line_noise_harmonics):
    meg_data += line_noise_scale * \
        line_noise_ampl_MEGs[:,None] * np.sin(2*np.pi * lnh * times)  [None,:]
    line_noise_scale *= line_noise_scale_decay

intensities = np.random.uniform(low=0.7,high=1, size= len(roi_sensors) )

for rli,roi_label in enumerate(roi_sensors.keys()):
    #meg_chis_inds = [0,20]
    meg_chis_inds = roi_sensors[roi_label]
    print(roi_label,meg_chis_inds)
    for chi_ind in meg_chis_inds:
        # chi_ind -- index of an element in meg_chis list (i.e. index of an index)
        trem_burst_meg = np.sin(2*np.pi * trem_burst_meg_freq * times)
        src_intensity =  intensities[rli]
        meg_data[chi_ind] += 2*meg_std* trem_burst_meg * tremor_intensity * src_intensity

In [ ]:
artfiacts_intervals_MEG

In [ ]:
# add artifacts to all sensors
for ai in artfiacts_intervals_MEG:
    start,end,at = ai
    start_bin,end_bin = int(start*sfreq), int(end*sfreq)
    sz = (nmegs,end_bin-start_bin)
    if at == 0:
        art_signal = np.random.uniform(low = -1, high = 1, size=sz) \
            * 2*meg_stds[:,None] * 4
    elif at == 1:
        art_signal = 2*meg_stds[:,None] * 4 * np.ones(sz)

    meg_data[:,start_bin:end_bin] += art_signal

In [ ]:
meg_chis

In [ ]:
meg_chis_inds, np.array(f.ch_names)[meg_chis][meg_chis_inds] 

In [ ]:
len(sens_types)

In [ ]:
roi_sensors

In [ ]:
sens_types = list( set(sensor_types) )
#fig,axs = plt.subplots(nrows=len(sens_types))
#for sti,st in enumerate(sens_types):
for chi_ind in list(roi_sensors[roi_labels_with_tremor[1]]):
    
    chi = meg_chis[chi_ind]
    chan_info = f.info['chs'][chi]
    
    #t = chan_info['coil_type']._name
    t = chan_info['coil_type']
    print(t)
    sti = sens_types.index( t )
    #print(chan_info['coil_type'])
    #ax = axs[sti]
    ax = plt.gca()
    d = meg_data[chi_ind]
    me = np.mean(d)
    ax.plot(times,d-me)

In [ ]:
plt.plot(times,meg_data[0])

In [ ]:
#f.ch_names

In [ ]:
print(len(meg_chis), meg_data.shape)
print(len(lfp_chis), lfp_data.shape)
print(len(emg_chis), emg_data.shape)

# Save result

In [ ]:
do_gen_MEG_data = 1

In [ ]:
new_chis = [];
if do_gen_MEG_data:
    new_chis = meg_chis
new_chis = new_chis + lfp_chis + emg_chis
new_chnames = np.array(f.ch_names)[new_chis].tolist()

print( len(new_chis), len(new_chnames) )

new_chtypes = len(new_chis)*[0]
for chii,chi in enumerate(new_chis):
    new_chtypes[chii] = mne.channel_type(info,chi)

if do_gen_MEG_data:
    dd = [meg_data, lfp_data, emg_data]
else:
    dd = [lfp_data, emg_data]
new_data = np.vstack(dd)
print(new_data.shape)

#new_chtypes

#new_chnames

new_info = mne.create_info(
    ch_names=new_chnames,
    ch_types=new_chtypes,
    sfreq=sfreq)

custom_raw_cur = mne.io.RawArray(new_data, new_info)

## high freq LFP

In [ ]:
new_chis_hires = lfp_chis
new_chnames_hires = np.array(f.ch_names)[new_chis_hires].tolist()

new_chtypes_hires = len(new_chis_hires)*[0]
for chii,chi in enumerate(new_chis_hires):
    new_chtypes_hires[chii] = mne.channel_type(info,chi)

new_data_hires = np.vstack([lfp_data_hires])
print(new_data_hires.shape)

#new_chtypes

#new_chnames

new_info_hires = mne.create_info(
    ch_names=new_chnames_hires,
    ch_types=new_chtypes_hires,
    sfreq=sfreq_high)

custom_raw_cur_hires = mne.io.RawArray(new_data_hires, new_info_hires)

### set coorect sensor locations (necessary for artifact finding)

In [ ]:
for chi,chn in enumerate(custom_raw_cur.ch_names):
    chi_orig = f.ch_names.index(chn)
    custom_raw_cur.info['chs'][chi] = f.info['chs'][chi_orig]
    
for chi,chn in enumerate(custom_raw_cur_hires.ch_names):
    chi_orig = f.ch_names.index(chn)
    custom_raw_cur_hires.info['chs'][chi] = f.info['chs'][chi_orig]

# Some parts of resave.py

In [ ]:
#rawname_new = 'S99_off_move'

## save highres LFP

In [ ]:
fname_noext_new_hires = rawname_new + '_LFP_1kHz.fif'
fname_new_hires = pjoin(gv.data_dir,fname_noext_new_hires)
custom_raw_cur_hires.save(fname_new_hires, overwrite=True)

### save lowres LFP and EMG

In [ ]:
import importlib as il
il.reload(upre)

In [ ]:
#%debug
upre.saveLFP(rawname_new, skip_if_exist = 0,sfreq=sfreq, raw_resampled=custom_raw_cur)
upre.extractEMGData(custom_raw_cur,rawname_new, skip_if_exist = 0)  

In [ ]:
rr = upre.getSubRaw(None,'LFP.*',raw=custom_raw_cur)

In [ ]:
len(rr.ch_names)

In [ ]:
custom_raw_cur.ch_names

In [ ]:
mne.pick_channels_regexp(custom_raw_cur.ch_names, 'LFP.*' )

In [ ]:
# I don't want to call resave.py directly because 
# it would require saving in FieldTrip format

In [ ]:
custom_raw_cur_notched = custom_raw_cur.copy()
custom_raw_cur_notched.notch_filter(line_noise_harmonics, n_jobs=6)
                                   #filter_length=sfreq, trans_bandwidth=1, 
                                   #notch_widths=1)

In [ ]:
#custom_raw_cur_notched.ch_names

In [ ]:
help(filt_raw.filter)

In [ ]:
help(np.pad)

In [ ]:
#%debug
MEG_artif_thr_mult = 2.5
MEG_thr_use_mean = 1
filt_raw = custom_raw_cur_notched.copy()
#filt_raw = custom_raw_cur.copy()
filt_raw.load_data().filter(l_freq=1., h_freq=None, pad='symmetric')

In [ ]:
custom_raw_cur

In [ ]:
#############################  Plot
anns, cvl_per_side = utils.findRawArtifacts(custom_raw_cur , 
                        thr_mult = MEG_artif_thr_mult,
    thr_use_mean = MEG_thr_use_mean)
filt_raw.set_annotations(anns)

filt_raw.annotations.save(os.path.join(gv.data_dir, 
                                       '{}_ann_MEGartif.txt'.format(rawname_new) ) )
print('Artif found ',anns)


#custom_raw_cur.filter(l_freq=1.,h_freq=None)

In [ ]:
help(filt_raw.plot)

In [ ]:
%matplotlib qt
filt_raw.plot(scalings='auto');

### 3D plots

In [ ]:
import mayavi.mlab as mam

meg_only = custom_raw_cur.copy()
meg_only.pick_types(meg=True)

res,res_inds = utils.collectChnamesBySide(meg_only.info)

meg_chis = mne.pick_channels_regexp(meg_only.ch_names,'MEG.*')

chlocs = []
for si in range(len(meg_chis)):
    chi = meg_chis[si]
    chan_info = meg_only.info['chs'][chi]
    chan_loc = chan_info['loc'][:3]
    chlocs += [chan_loc]
chlocs = np.vstack(chlocs)

sensors_inds = []
src_inds = []
sides = ['left']
for roi_label in sides:
    sensors_inds +=  list(res_inds[roi_label] ) 
    

x,y,z = chlocs.T * 100
mam.points3d(x,y,z, scale_factor=0.3, color=(0.1,0.1,0.1))#, color = tuple(clrs[grpi]) )

x,y,z = chlocs[sensors_inds].T * 100
mam.points3d(x,y,z, scale_factor=0.5)#, color = tuple(clrs[grpi]) )

sensors_inds = []
for roi_label in roi_sensors:    
    src_inds += list(np.where(srcgroups_dict[sgdn] == labels_dict[sgdn].index(roi_label))[0])
    sensors_inds +=  list(roi_sensors[roi_label] ) 
    
x,y,z = chlocs[sensors_inds].T * 100
mam.points3d(x,y,z, scale_factor=0.7, color=(1,0,0))#, color = tuple(clrs[grpi]) )

x,y,z = coords[src_inds].T 
mam.points3d(x,y,z, scale_factor=0.5, color=(1,1,0))#, color = tuple(clrs[grpi]) )


In [ ]:
mam.show()

In [ ]:
#mam.show()
mam.close()

## Final check

In [ ]:
save_filt = 0
if save_filt:
    raw_to_save = filt_raw
else:
    raw_to_save = custom_raw_cur

In [ ]:
%matplotlib inline

In [ ]:
chns =  [emg_chnames_with_tremor[0],  info.ch_names[glob_index_lfp], 
         info.ch_names[ meg_chis[ roi_sensors[roi_labels_with_tremor[0]][0] ] ] ]
for chn in chns:
    dat = raw_to_save[chn][0][0]
    plt.plot(times,dat)
    plt.title(chn)
    plt.show()

In [ ]:
#save raw
if save_filt:
    fname_noext_new = rawname_new + '_resample_notch_highpass_raw.fif'
else:
    fname_noext_new = rawname_new + '_resample_raw.fif'
fname_new = pjoin(gv.data_dir,fname_noext_new)
raw_to_save.save(fname_new, overwrite=True)

In [ ]:
# Save supp test file info
supp_info_fname_noext = rawname_new + '_supp_info'
supp_info_fname = supp_info_fname_noext + '.npz'
supp_info_fname_full = pjoin(gv.data_dir,supp_info_fname)
np.savez(supp_info_fname_full,supp_info=supp_info)

import scipy.io as sio
sio.savemat(pjoin(gv.data_dir,supp_info_fname_noext + '.mat'), supp_info)

In [ ]:
supp_info.keys()

In [ ]:
supp_info

In [ ]:
supp_info['artfiacts_intervals_LFP']

In [ ]:
anns_artif_LFP = utils.intervals2anns(supp_info['artfiacts_intervals_LFP'],'BAD_LFP{}'.format(opsidelet))

In [ ]:
anns_artif_LFP.save(pjoin(gv.data_dir,'{}_ann_LFPartif.txt'.format(rawname_new)))

In [ ]:
%run -i run_collect_artifacts.py -r S97_off_move --min_dur 5

In [ ]:
ann_list

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%run -i run_process_FTsources.py -r S97_off_move --groupings all_raw --alg_type all_sources

In [ ]:
ivalis_artif_tb_indarrays_merged